- 1.商品信息部分

字段处理：

商品名称：分词统计，jieba

所属类别：拆分后与商品类别列表关联，onehot编码

In [3]:
# 分词统计:
import jieba
from jieba import analyse
def analyse_text(text):
    analyse.set_stopwords('') #此处加载停用词表
    split_words = analyse.extract_tags(text)
    return split_words

# 独热编码
def onehot_code(word_list):
    return ','.join(word_list).str.get_dummies(sep=',')
    

- 2.商品类别列表

与商品信息列表进行关联

- 3.训练集

数据id：检查唯一性

用户id：聚合键，一些统计指标

商品id：关联键，上述2个文件处理完之后关联到文件中

评论时间戳： 按照时间戳对评论进行排序和赋权

评论标题： 分词进行情感分析

评论内容：nlp进行情感分析，词频统计

完成之后进行交叉验证

In [5]:
# 唯一性检查
train['数据ID'].unique() == len(train['数据ID'])

#评论时间戳:


# 评论内容：
# 评论长度
stopwords = 
def words_count(text):
    sub_replace = re.compile('[^0-9a-z\s]')
    text = sub_replace.sub('',text.lower())
    text = ' 'join(word for word in text if word not in stopwords)
    return len(text)


# 数据清洗，出去无用字符，此处需检查
def standardize_text(df, text_field):
    df[text_field] = df[text_field].str.replace(r"http\S+", "")
    df[text_field] = df[text_field].str.replace(r"http", "")
    df[text_field] = df[text_field].str.replace(r"@\S+", "")
    df[text_field] = df[text_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[text_field] = df[text_field].str.replace(r"@", "at")
    df[text_field] = df[text_field].str.lower()
    return df

# 简单的文本情感分析
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=22, shuffle=True)
def text_emotion_analyse(data,text):
    vectorizer = CountVectorizer()
    words_features = vectorizer.fit_transform(data[text]).toarray
    return words_features
       

SyntaxError: invalid syntax (<ipython-input-5-7650bfbee4bd>, line 9)

In [ ]:
# 参数寻优：RandomGridSearchCV+lightgbm
from sklearn.model_selection import RandomizedSearchCV
# from sklearn.model_selection import LGBMClassifier
import lightgbm as lgb
parameters = {
    'max_depth': [4,6,8],
    'num_leaves': [20,30,40],
}

gbm = lgb.LGBMClassifier(objective = 'binary',
                         is_unbalance = True,
                         metric = 'binary_logloss,auc',
                         max_depth = 6,
                         num_leaves = 40,
                         learning_rate = 0.1,
                         feature_fraction = 0.7,
                         min_child_samples=21,
                         min_child_weight=0.001,
                         bagging_fraction = 1,
                         bagging_freq = 2,
                         reg_alpha = 0.001,
                         reg_lambda = 8,
                         cat_smooth = 0,
                         num_iterations = 200,   
                        )
gsearch = RandomizedSearchCV(gbm, param_grid=parameters, scoring='roc_auc', cv=3)
gsearch.fit(train_features, train_y)
print('参数的最佳取值:{0}'.format(gsearch.best_params_))
print('最佳模型得分:{0}'.format(gsearch.best_score_))
print(gsearch.cv_results_['mean_test_score'])
print(gsearch.cv_results_['params'])

In [ ]:
# 交叉验证
from sklearn.model_selection import KFold
from sklearn.metrics import 
kf = KFold(n_splits=5, random_state=22, shuffle=True)
def data_split_vaild():
    scores = []
    for train_idx, eval_idx in kf.split(features,target):
        clf.fit(features[train_idx],target[train_idx].values)
        predictions = clf.predict(features[eval_idx])
        